In [1]:
import numpy as np
import pandas as pd
import re

### 1. clean data

In [51]:
# 读取数据
data = pd.read_csv("Data.csv", encoding='latin1')


#### 1-1 格式处理

In [52]:
# 处理列名

## 去除列名首尾的space
data.columns = data.columns.str.strip()
## 去除名中的特殊字符为_
data.columns = (
    data.columns
      .str.lower()  # 转小写
      .str.replace(r"[^a-z0-9 _]", " ", regex=True)  # 保留小写字母、数字、空格、_
      .str.replace(r"\s+", "_", regex=True)  # 多空格变_
      .str.strip("_")  # 去除开头结尾的_
)

data.columns


Index(['contact_id', 'intake_details_submission_date',
       'type_of_assistance_applied', 'care_team', 'gender', 'age', 'race',
       'occupation', 'housing_type', 'home_ownership',
       'monthly_mortgage_cpf_cash', 'monthly_rental', 'identity_id_type',
       'intake_no_of_hh', 'intake_pci', 'intake_status',
       'reason_for_rejection_or_cancelled', 'income_assessment_salary',
       'income_assessment_cpf_payout',
       'income_assessment_assistance_from_other_agencies',
       'income_assessment_assistance_from_relatives_friends',
       'income_assessment_insurance_payout', 'income_assessment_rental_income',
       'income_assessment_others_income', 'income_assessment_total_income',
       'expenditure_assessment_mortgage_rental',
       'expenditure_assessmen_utilities', 'expenditure_assessment_s_cc_fees',
       'expenditure_assessment_food_expenses',
       'expenditure_assessment_marketing_groceries',
       'expenditure_assessment_telecommunications',
       'expenditu

In [53]:
# 处理date列
data['intake_details_submission_date'] = pd.to_datetime(data['intake_details_submission_date'])
data['assessment_date_time'] = pd.to_datetime(data['assessment_date_time'])


In [54]:
data.dtypes

contact_id                                                       int64
intake_details_submission_date                          datetime64[ns]
type_of_assistance_applied                                      object
care_team                                                       object
gender                                                          object
age                                                            float64
race                                                            object
occupation                                                      object
housing_type                                                    object
home_ownership                                                  object
monthly_mortgage_cpf_cash                                       object
monthly_rental                                                  object
identity_id_type                                                object
intake_no_of_hh                                                float64
intake

In [55]:
# 去除格式为object列的特殊字符
def clean_text(text):
    if isinstance(text, str):
        # 移除非字母或数字的字符
        text = text.replace("?","")
        # 转为小写
        text = text.lower()
    return text

for col in data.columns:
    if data[col].dtype == 'object':
        data[col] = data[col].apply(clean_text)

In [56]:
# 去除格式为numeric列的特殊字符
def clean_dollar(text):
    if isinstance(text, str):
        text = text.replace("(","-")
        # 使用正则表达式保留数字、小数点和负号
        text = re.sub(r"[^\d.-]", "", text)
        
        # 检查结果是否是一个有效的数字
        # 如果结果是一个单独的负号或空字符串，则返回空字符串
        if text in ["", "-"]:
            return ""
        
        # 确保负号只出现在开头
        if text.count('-') > 1 or (text.count('-') == 1 and text[0] != '-'):
            return ""
        
    return text


dollar_cols = [
    "assistance_amount",
    "copayment",
    "monthly_rental",
    "i_e_difference_self_declaration",
    "i_e_difference_assessment",
    "invoice_amount",
]

for col in dollar_cols:
    data[col] = data[col].apply(clean_dollar)
    data[col] = pd.to_numeric(data[col])

In [57]:
# 处理occupation，仅保留'unemployed','retired','part-time','student'四种
occupation = {
    "unemployed": [
        "unfit for work",
        "unemployed",
        "unemployed unfit for work",
        "unemployedunfit for work",
        "unemployedmedically unfit to work",
        "unemployed (medically unfit for work)",
        "unemployed (paraplegic since 2005 accident)",
        "unemployedmedically unfit for work",
        "unempolyed",
        "not employed",
        "not working",
        "nil",
        "unemployed (due to medical conditions)",
        "unemployed/ can collector",
        "unfit for work",
        "not sure",
        "uneployed",
        "unempployed",
        "unemployed due to medical reasons",
        "unemployed (unfit for work)",
        "unemployed (unfit for work)",
        "unemployed (elderly)",
        "unemployed (medically unfit)",
        "unfit to work",
        "unfit for employment",
        "n.a.",
        "unemployed- permantly unfit for work",
        "unemployed due to medical conditions",
        "not been employed for the past 7 years",
    ],
    "retired": [
        "retired",
        "retiree",
        "tetiree",
        "reitree",
        "retired",
        "retired cleaner",
        "na - retiree",
        "retired cleaner",
        "retiree",
    ],
    "part-time": [
        "part timer",
        "part-time actor",
        "part time baker",
        "part time f&b",
        "part-time tutor",
        "part-time lala move driver",
        "part-timer",
        "part-time admin",
        "part-time service crew",
        "part-time security officer",
        "part time macdonald",
        "part time security",
        "part-time cashier",
        "part-time shop assistant",
        "part-time security guard",
        "part-time cleaner",
        "part-time stall assistant",
        "part-time",
        "part-time photo developer",
        "part time kitchen helper",
        "part-timer art teacher",
    ],
    "student": ["student", "nus y2 student"],
}


employed_set = set()


def unify_occupation(text):
    global employed_set
    if pd.isna(text) or text.strip() == "":
        return "NA"
    find = False
    # print(text)
    text = text.strip()
    ans = ""
    for i in occupation:
        if text in occupation[i]:
            find = True
            ans = i
            break

    if find:
        return ans
    else:
        employed_set.add(text)
        return "employed"


data["occupation"] = data["occupation"].apply(unify_occupation)

# 分类计数 Type of Assistance Applied 列
occupation_counts = data["occupation"].value_counts()

# 打印结果
print(occupation_counts)


occupation
unemployed    799
employed      258
retired       230
NA            101
student        34
part-time      24
Name: count, dtype: int64


#### 1-2 删除与目标y无关列

In [58]:
# drop一些无法使用的列
drop_cols_1=[
    "contact_id",
    "intake_details_submission_date",
    "reason_for_rejection_or_cancelled",
    "assistance_details",
    "case_profile",
    "remarks_no_of_hh"
]

# drop重复的列，applicant自己报的和内部评估的之间只取内部评估的
drop_cols_2=[
    "monthly_mortgage_cpf_cash",
    'monthly_rental',
    'intake_pci'
]

# 根据slides有一些features是options
# -- ‘Nationality', 'ID type', 'Gender', 'Race', 'type of assistant applied', 'Housing Type', 'Home Ownership'
# 删除其中无关项

drop_cols_3=[
    'identity_id_type',
    'gender', 
    'race', 
    'housing_type', 
    'home_ownership'
]


drop_cols = drop_cols_1 + drop_cols_2 + drop_cols_3
data = data.drop(columns=drop_cols)


In [59]:
data

,type_of_assistance_applied,care_team,age,occupation,intake_no_of_hh,intake_status,income_assessment_salary,income_assessment_cpf_payout,income_assessment_assistance_from_other_agencies,income_assessment_assistance_from_relatives_friends,...,assistance_amount,invoice_amount,copayment,no_of_hh,before_primary,primary_7_12,secondary_13_17,tertiary_18_21,adult_22_64,elderly_65_and_above
0,education fees assistance,central 1 1,13.0,student,3.0,confirm,0.0,0.0,500.0,0.0,...,900.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,interim dialysis assistance,central 1 1,57.0,unemployed,2.0,confirm,1500.0,0.0,0.0,0.0,...,1144.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,medical consumables assistance,central 1 1,32.0,unemployed,4.0,confirm,3380.0,0.0,0.0,500.0,...,243.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,household living assistance,central 1 1,69.0,NA,2.0,confirm,525.0,0.0,200.0,400.0,...,243.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,household living assistance,central 1 1,70.0,NA,1.0,confirm,134.0,0.0,90.0,0.0,...,200.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,education fees assistance,south 1 1,14.0,student,NaN,NaN,2400.0,0.0,240.0,0.0,...,571.85,NaN,NaN,3.0,NaN,NaN,1.0,NaN,1.0,1.0
1442,household living assistance,central 1 1,37.0,part-time,NaN,NaN,875.0,0.0,1080.0,0.0,...,200.00,NaN,NaN,4.0,1.0,1.0,NaN,NaN,2.0,NaN
1443,education fees assistance,south 3 3,19.0,student,NaN,NaN,950.0,0.0,2500.6,400.0,...,3321.20,NaN,NaN,7.0,1.0,1.0,NaN,2.0,3.0,NaN
1444,medical transport assistance,east 4 4,67.0,retired,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2080.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0


In [60]:
data.columns

Index(['type_of_assistance_applied', 'care_team', 'age', 'occupation',
       'intake_no_of_hh', 'intake_status', 'income_assessment_salary',
       'income_assessment_cpf_payout',
       'income_assessment_assistance_from_other_agencies',
       'income_assessment_assistance_from_relatives_friends',
       'income_assessment_insurance_payout', 'income_assessment_rental_income',
       'income_assessment_others_income', 'income_assessment_total_income',
       'expenditure_assessment_mortgage_rental',
       'expenditure_assessmen_utilities', 'expenditure_assessment_s_cc_fees',
       'expenditure_assessment_food_expenses',
       'expenditure_assessment_marketing_groceries',
       'expenditure_assessment_telecommunications',
       'expenditure_assessment_transportation',
       'expenditure_assessmen_medical_expenses',
       'expenditure_assessment_education_expense',
       'expenditure_assessmen_contribution_to_family_members',
       'expenditure_assessment_domestic_helper',
   

#### 1-3 复核原表中的数据是否有计算问题
#### -- 计算有问题，删除手动计算列 （‘Income Assessment: Total Income’， 'Expenditure Assessment: Total Expenditure'， 'I&E Difference (self-declaration)', 'I&E Difference (assessment)'）

In [61]:
# 填充所有空白值为0
def fill_0(text):
    if isinstance(text, str) and text.strip() == "":
        return 0
    return text


data = data.apply(fill_0)  # 先处理空白填0
data = data.fillna(0)  # 万一有非法的NaN也一起补成0
data


,type_of_assistance_applied,care_team,age,occupation,intake_no_of_hh,intake_status,income_assessment_salary,income_assessment_cpf_payout,income_assessment_assistance_from_other_agencies,income_assessment_assistance_from_relatives_friends,...,assistance_amount,invoice_amount,copayment,no_of_hh,before_primary,primary_7_12,secondary_13_17,tertiary_18_21,adult_22_64,elderly_65_and_above
0,education fees assistance,central 1 1,13.0,student,3.0,confirm,0.0,0.0,500.0,0.0,...,900.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,interim dialysis assistance,central 1 1,57.0,unemployed,2.0,confirm,1500.0,0.0,0.0,0.0,...,1144.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,medical consumables assistance,central 1 1,32.0,unemployed,4.0,confirm,3380.0,0.0,0.0,500.0,...,243.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,household living assistance,central 1 1,69.0,NA,2.0,confirm,525.0,0.0,200.0,400.0,...,243.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,household living assistance,central 1 1,70.0,NA,1.0,confirm,134.0,0.0,90.0,0.0,...,200.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,education fees assistance,south 1 1,14.0,student,0.0,0,2400.0,0.0,240.0,0.0,...,571.85,0.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,1.0
1442,household living assistance,central 1 1,37.0,part-time,0.0,0,875.0,0.0,1080.0,0.0,...,200.00,0.0,0.0,4.0,1.0,1.0,0.0,0.0,2.0,0.0
1443,education fees assistance,south 3 3,19.0,student,0.0,0,950.0,0.0,2500.6,400.0,...,3321.20,0.0,0.0,7.0,1.0,1.0,0.0,2.0,3.0,0.0
1444,medical transport assistance,east 4 4,67.0,retired,0.0,0,0.0,0.0,0.0,0.0,...,0.00,2080.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [62]:
data['income_total_cal'] = data['income_assessment_salary'] + data['income_assessment_cpf_payout'] + data['income_assessment_assistance_from_other_agencies'] + data['income_assessment_assistance_from_relatives_friends'] + data['income_assessment_insurance_payout'] + data['income_assessment_rental_income'] + data['income_assessment_others_income']
data['expenditure_total_cal'] = data['expenditure_assessment_mortgage_rental'] + data['expenditure_assessmen_utilities'] + data['expenditure_assessment_s_cc_fees'] + data['expenditure_assessment_food_expenses'] + data['expenditure_assessment_marketing_groceries'] + data['expenditure_assessment_telecommunications'] + data['expenditure_assessment_transportation'] + data['expenditure_assessmen_medical_expenses'] + data['expenditure_assessment_education_expense'] + data['expenditure_assessmen_contribution_to_family_members'] + data['expenditure_assessment_domestic_helper'] + data['expenditure_assessment_loans_debts_installments'] + data['expenditure_assessment_insurance_premiums'] + data['expenditure_assessment_others_expenditure']
data['difference_cal'] = data['income_total_cal'] - data['expenditure_total_cal']
data


,type_of_assistance_applied,care_team,age,occupation,intake_no_of_hh,intake_status,income_assessment_salary,income_assessment_cpf_payout,income_assessment_assistance_from_other_agencies,income_assessment_assistance_from_relatives_friends,...,no_of_hh,before_primary,primary_7_12,secondary_13_17,tertiary_18_21,adult_22_64,elderly_65_and_above,income_total_cal,expenditure_total_cal,difference_cal
0,education fees assistance,central 1 1,13.0,student,3.0,confirm,0.0,0.0,500.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0,1601.90,-1101.90
1,interim dialysis assistance,central 1 1,57.0,unemployed,2.0,confirm,1500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500.0,1084.00,416.00
2,medical consumables assistance,central 1 1,32.0,unemployed,4.0,confirm,3380.0,0.0,0.0,500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3880.0,3494.00,386.00
3,household living assistance,central 1 1,69.0,NA,2.0,confirm,525.0,0.0,200.0,400.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1125.0,1114.00,11.00
4,household living assistance,central 1 1,70.0,NA,1.0,confirm,134.0,0.0,90.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,474.0,700.00,-226.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,education fees assistance,south 1 1,14.0,student,0.0,0,2400.0,0.0,240.0,0.0,...,3.0,0.0,0.0,1.0,0.0,1.0,1.0,2640.0,2701.25,-61.25
1442,household living assistance,central 1 1,37.0,part-time,0.0,0,875.0,0.0,1080.0,0.0,...,4.0,1.0,1.0,0.0,0.0,2.0,0.0,1955.0,1904.60,50.40
1443,education fees assistance,south 3 3,19.0,student,0.0,0,950.0,0.0,2500.6,400.0,...,7.0,1.0,1.0,0.0,2.0,3.0,0.0,3850.6,2418.60,1432.00
1444,medical transport assistance,east 4 4,67.0,retired,0.0,0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.00,0.00


In [63]:
data['income_total_check'] = data['income_total_cal'] - data['income_assessment_total_income']
data['expenditure_total_check'] = data['expenditure_total_cal'] - data['expenditure_assessment_total_expenditure']
data['difference_check'] = data['i_e_difference_assessment'] - data['difference_cal']
income_nonzero_indices = data.index[data["income_total_check"] != 0].tolist()
expenditure_nonzero_indices = data.index[data["expenditure_total_check"] != 0].tolist()
difference_nonzero_indices = data.index[data["difference_check"] != 0].tolist()

print(len(income_nonzero_indices))
print(len(expenditure_nonzero_indices))
print(len(difference_nonzero_indices))
# 原本手动计算有错误，所以删除手动计算的列，替换为code计算结果

84
241
584


##### 随便找了组数据看为什么有差异

In [64]:
data['difference_check'].sum()

132616.53999999998

In [65]:
check_indices = data.index[data["difference_check"] == data['difference_check'].max()].tolist()
check_indices

[1094]

In [66]:
data.loc[check_indices]

,type_of_assistance_applied,care_team,age,occupation,intake_no_of_hh,intake_status,income_assessment_salary,income_assessment_cpf_payout,income_assessment_assistance_from_other_agencies,income_assessment_assistance_from_relatives_friends,...,secondary_13_17,tertiary_18_21,adult_22_64,elderly_65_and_above,income_total_cal,expenditure_total_cal,difference_cal,income_total_check,expenditure_total_check,difference_check
1094,medical consumables assistance,central 1 1,51.0,unemployed,5.0,confirm,0.0,0.0,1837.0,0.0,...,0.0,0.0,4.0,1.0,1837.0,7601.4,-5764.4,0.0,300.0,11228.8


##### 删除无用列

In [67]:
drop_cols=[
'income_assessment_total_income',
'expenditure_assessment_total_expenditure',
'i_e_difference_self_declaration', 
'i_e_difference_assessment',
'income_total_check',
'expenditure_total_check',
'difference_check'
]

data = data.drop(columns=drop_cols)
print(data.columns)

Index(['type_of_assistance_applied', 'care_team', 'age', 'occupation',
       'intake_no_of_hh', 'intake_status', 'income_assessment_salary',
       'income_assessment_cpf_payout',
       'income_assessment_assistance_from_other_agencies',
       'income_assessment_assistance_from_relatives_friends',
       'income_assessment_insurance_payout', 'income_assessment_rental_income',
       'income_assessment_others_income',
       'expenditure_assessment_mortgage_rental',
       'expenditure_assessmen_utilities', 'expenditure_assessment_s_cc_fees',
       'expenditure_assessment_food_expenses',
       'expenditure_assessment_marketing_groceries',
       'expenditure_assessment_telecommunications',
       'expenditure_assessment_transportation',
       'expenditure_assessmen_medical_expenses',
       'expenditure_assessment_education_expense',
       'expenditure_assessmen_contribution_to_family_members',
       'expenditure_assessment_domestic_helper',
       'expenditure_assessment_loans_

In [68]:
data

,type_of_assistance_applied,care_team,age,occupation,intake_no_of_hh,intake_status,income_assessment_salary,income_assessment_cpf_payout,income_assessment_assistance_from_other_agencies,income_assessment_assistance_from_relatives_friends,...,no_of_hh,before_primary,primary_7_12,secondary_13_17,tertiary_18_21,adult_22_64,elderly_65_and_above,income_total_cal,expenditure_total_cal,difference_cal
0,education fees assistance,central 1 1,13.0,student,3.0,confirm,0.0,0.0,500.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0,1601.90,-1101.90
1,interim dialysis assistance,central 1 1,57.0,unemployed,2.0,confirm,1500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500.0,1084.00,416.00
2,medical consumables assistance,central 1 1,32.0,unemployed,4.0,confirm,3380.0,0.0,0.0,500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3880.0,3494.00,386.00
3,household living assistance,central 1 1,69.0,NA,2.0,confirm,525.0,0.0,200.0,400.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1125.0,1114.00,11.00
4,household living assistance,central 1 1,70.0,NA,1.0,confirm,134.0,0.0,90.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,474.0,700.00,-226.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,education fees assistance,south 1 1,14.0,student,0.0,0,2400.0,0.0,240.0,0.0,...,3.0,0.0,0.0,1.0,0.0,1.0,1.0,2640.0,2701.25,-61.25
1442,household living assistance,central 1 1,37.0,part-time,0.0,0,875.0,0.0,1080.0,0.0,...,4.0,1.0,1.0,0.0,0.0,2.0,0.0,1955.0,1904.60,50.40
1443,education fees assistance,south 3 3,19.0,student,0.0,0,950.0,0.0,2500.6,400.0,...,7.0,1.0,1.0,0.0,2.0,3.0,0.0,3850.6,2418.60,1432.00
1444,medical transport assistance,east 4 4,67.0,retired,0.0,0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.00,0.00


#### 1-4 将表中的assistance type转换为甲方的标准type
#### -- 使用内部评估版，如内部评估版无数据，则按applied的种类

In [ ]:
# household living assistance
# HIV medication assistance
# interim dialysis assistance 透析（仅提供3个月资助）
# day care fees assistance
# medical transport assistance 提供1年的交通费
# antenatal check fees 产检
# medical consumables assistance 医疗耗材
# education fees assistance
# HCV medication fees assistance 丙肝（一次性药物费用）
# others


In [69]:
# 用type_of_assistance_applied列补齐type_of_assistances列缺失值
data.loc[data["type_of_assistances"] == 0, "type_of_assistances"] = data.loc[data["type_of_assistances"] == 0, "type_of_assistance_applied"]

# 处理 type_of_assistances
data["type_of_assistances"] = data["type_of_assistances"].apply(lambda x:x.split(",")[0].strip())

data["type_of_assistances"] = (
    data["type_of_assistances"]
      .str.lower()  # 转小写
      .str.replace(r"[^a-z _]", " ", regex=True)  # 保留小写字母、空格、_
      .str.replace(r"\s+", "_", regex=True)  # 多空格变_
      .str.strip("_")  # 去除开头结尾的_
)

# 分类计数 type_of_assistance_applied 列
assistance_counts = data['type_of_assistances'].value_counts()

# 打印结果
print(assistance_counts)

type_of_assistances
interim_dialysis_assistance        578
medical_consumables_assistance     340
medical_transport_assistance       209
household_living_assistance        198
education_fees_assistance           32
hiv_medication_fees                 31
day_care_fees_assistance            19
befriending_service                 17
antenatal_check_fees_assistance     11
one_time_assistance                  7
others                               4
Name: count, dtype: int64


In [70]:
data.loc[data["type_of_assistances"].isin(["befriending_service", "one_time_assistance"]), "type_of_assistances"] = "others"
assistance_counts = data['type_of_assistances'].value_counts()

print(assistance_counts)

type_of_assistances
interim_dialysis_assistance        578
medical_consumables_assistance     340
medical_transport_assistance       209
household_living_assistance        198
education_fees_assistance           32
hiv_medication_fees                 31
others                              28
day_care_fees_assistance            19
antenatal_check_fees_assistance     11
Name: count, dtype: int64


In [71]:
# drop type_of_assistance_applied列
data = data.drop(columns='type_of_assistance_applied')
data

,care_team,age,occupation,intake_no_of_hh,intake_status,income_assessment_salary,income_assessment_cpf_payout,income_assessment_assistance_from_other_agencies,income_assessment_assistance_from_relatives_friends,income_assessment_insurance_payout,...,no_of_hh,before_primary,primary_7_12,secondary_13_17,tertiary_18_21,adult_22_64,elderly_65_and_above,income_total_cal,expenditure_total_cal,difference_cal
0,central 1 1,13.0,student,3.0,confirm,0.0,0.0,500.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0,1601.90,-1101.90
1,central 1 1,57.0,unemployed,2.0,confirm,1500.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500.0,1084.00,416.00
2,central 1 1,32.0,unemployed,4.0,confirm,3380.0,0.0,0.0,500.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3880.0,3494.00,386.00
3,central 1 1,69.0,NA,2.0,confirm,525.0,0.0,200.0,400.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1125.0,1114.00,11.00
4,central 1 1,70.0,NA,1.0,confirm,134.0,0.0,90.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,474.0,700.00,-226.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,south 1 1,14.0,student,0.0,0,2400.0,0.0,240.0,0.0,0.0,...,3.0,0.0,0.0,1.0,0.0,1.0,1.0,2640.0,2701.25,-61.25
1442,central 1 1,37.0,part-time,0.0,0,875.0,0.0,1080.0,0.0,0.0,...,4.0,1.0,1.0,0.0,0.0,2.0,0.0,1955.0,1904.60,50.40
1443,south 3 3,19.0,student,0.0,0,950.0,0.0,2500.6,400.0,0.0,...,7.0,1.0,1.0,0.0,2.0,3.0,0.0,3850.6,2418.60,1432.00
1444,east 4 4,67.0,retired,0.0,0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.00,0.00


#### 1-5 删除被否记录

In [72]:
# 删掉“Intake_Status”为非“confirm”的行
data2 = data.copy()
data2 = data2[data2['intake_status'] == "confirm"]
# 删掉"Recommendation"为"rejected"的行
data2 = data2[data2['recommendation'].str.strip() != "rejected"]
#drop "intake_status""recommendation"两列

drop_cols=[
    "intake_status",
    "recommendation"
]

data2 = data2.drop(columns=drop_cols)

data2

,care_team,age,occupation,intake_no_of_hh,income_assessment_salary,income_assessment_cpf_payout,income_assessment_assistance_from_other_agencies,income_assessment_assistance_from_relatives_friends,income_assessment_insurance_payout,income_assessment_rental_income,...,no_of_hh,before_primary,primary_7_12,secondary_13_17,tertiary_18_21,adult_22_64,elderly_65_and_above,income_total_cal,expenditure_total_cal,difference_cal
0,central 1 1,13.0,student,3.0,0.0,0.00,500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500.00,1601.9,-1101.90
1,central 1 1,57.0,unemployed,2.0,1500.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500.00,1084.0,416.00
2,central 1 1,32.0,unemployed,4.0,3380.0,0.00,0.0,500.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3880.00,3494.0,386.00
3,central 1 1,69.0,NA,2.0,525.0,0.00,200.0,400.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1125.00,1114.0,11.00
4,central 1 1,70.0,NA,1.0,134.0,0.00,90.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,474.00,700.0,-226.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,south 1 1,55.0,unemployed,4.0,0.0,0.00,1780.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,1.0,2.0,0.0,1780.00,1730.0,50.00
1321,east 1 1,55.0,employed,1.0,0.0,0.00,600.0,150.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,750.00,2285.0,-1535.00
1326,south 2 2,62.0,unemployed,3.0,0.0,0.00,1080.0,0.0,0.0,500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1580.00,1530.0,50.00
1330,east 2 2,66.0,employed,0.0,2800.0,876.95,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,1.0,0.0,3.0,0.0,3676.95,2416.5,1260.45


#### 1-6 合并assistance amount & invoice amount 为目标y: amount_total, 删除copayment
#### -- 甲方解释两者相互独立，invoice amount 包含 copayment

In [73]:
data3 = data2.copy()
# 合并assistance_amount,invoice_amount,copayment
data3['amount_total'] = data3['assistance_amount'] + data3['invoice_amount']
drop_cols = ["assistance_amount", "invoice_amount", "copayment"]

data3 = data3.drop(columns=drop_cols)

# 删除目标变量amount_total中为 NaN 的行
data3.dropna(subset=['amount_total'], inplace=True)
data3

,care_team,age,occupation,intake_no_of_hh,income_assessment_salary,income_assessment_cpf_payout,income_assessment_assistance_from_other_agencies,income_assessment_assistance_from_relatives_friends,income_assessment_insurance_payout,income_assessment_rental_income,...,before_primary,primary_7_12,secondary_13_17,tertiary_18_21,adult_22_64,elderly_65_and_above,income_total_cal,expenditure_total_cal,difference_cal,amount_total
0,central 1 1,13.0,student,3.0,0.0,0.00,500.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,500.00,1601.9,-1101.90,900.00
1,central 1 1,57.0,unemployed,2.0,1500.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1500.00,1084.0,416.00,1144.90
2,central 1 1,32.0,unemployed,4.0,3380.0,0.00,0.0,500.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3880.00,3494.0,386.00,243.00
3,central 1 1,69.0,NA,2.0,525.0,0.00,200.0,400.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1125.00,1114.0,11.00,243.00
4,central 1 1,70.0,NA,1.0,134.0,0.00,90.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,474.00,700.0,-226.00,200.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,south 1 1,55.0,unemployed,4.0,0.0,0.00,1780.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2.0,0.0,1780.00,1730.0,50.00,717.20
1321,east 1 1,55.0,employed,1.0,0.0,0.00,600.0,150.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,750.00,2285.0,-1535.00,396.80
1326,south 2 2,62.0,unemployed,3.0,0.0,0.00,1080.0,0.0,0.0,500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1580.00,1530.0,50.00,126.00
1330,east 2 2,66.0,employed,0.0,2800.0,876.95,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,3.0,0.0,3676.95,2416.5,1260.45,1213.15


#### 1-7 导出data

In [74]:
data3.to_csv("Cleaned_Data_0502.csv", index=False)